-- ============================================================================
# SILVER LAYER - Cleaned and Enriched Data
-- ============================================================================

In [1]:
spark.sql("CREATE DATABASE IF NOT EXISTS silver")
spark.sql("SHOW DATABASES").show()

25/10/28 08:31:27 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---------+
|namespace|
+---------+
|   bronze|
|  default|
|     gold|
|   silver|
+---------+



In [2]:
def create_query_in_location(location, query):
    query += f"\n LOCATION 's3a://data/silver/{location}'"
    spark.sql(query).show()

In [3]:
transactions = """
CREATE TABLE IF NOT EXISTS silver.transactions (
    transaction_id BIGINT NOT NULL,
    customer_id BIGINT NOT NULL,
    transaction_timestamp TIMESTAMP,
    transaction_date DATE,
    channel STRING,
    store_id BIGINT,
    payment_method STRING,
    payment_status STRING,

    -- Amounts
    subtotal DECIMAL(12,2),
    tax_amount DECIMAL(12,2),
    shipping_cost DECIMAL(12,2),
    discount_amount DECIMAL(12,2),
    total_amount DECIMAL(12,2),
    currency STRING,

    -- Loyalty
    loyalty_points_earned INT,
    loyalty_points_redeemed INT,
    coupon_codes ARRAY<STRING>,

    -- Derived Fields
    transaction_year INT,
    transaction_month INT,
    transaction_quarter INT,
    transaction_day_of_week INT,
    transaction_hour INT,
    is_weekend BOOLEAN,
    is_holiday BOOLEAN,
    fiscal_year INT,
    fiscal_quarter INT,

    -- Business Flags
    is_first_purchase BOOLEAN,
    is_return_transaction BOOLEAN,
    has_discount BOOLEAN,
    has_coupon BOOLEAN,

    -- Metadata
    source_system STRING,
    created_timestamp TIMESTAMP,
    updated_timestamp TIMESTAMP
)
USING PARQUET
PARTITIONED BY (transaction_year, transaction_month)
"""
create_query_in_location(query=transactions, location="transactions")

25/10/28 08:31:29 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


++
||
++
++



In [4]:
transaction_items = """
CREATE TABLE IF NOT EXISTS silver.transaction_items (
    transaction_item_id BIGINT NOT NULL,
    transaction_id BIGINT NOT NULL,
    product_id BIGINT NOT NULL,
    variant_id BIGINT,

    -- Quantities
    quantity INT,
    return_quantity INT,
    net_quantity INT,

    -- Amounts
    unit_price DECIMAL(10,2),
    unit_cost DECIMAL(10,2),
    discount_percentage DECIMAL(5,2),
    tax_rate DECIMAL(5,4),
    line_subtotal DECIMAL(12,2),
    line_discount DECIMAL(12,2),
    line_tax DECIMAL(12,2),
    line_total DECIMAL(12,2),
    line_cost DECIMAL(12,2),
    line_profit DECIMAL(12,2),
    line_margin_percentage DECIMAL(5,2),

    -- Fulfillment
    return_reason STRING,
    fulfillment_status STRING,
    warehouse_id BIGINT,

    -- Metadata
    source_system STRING,
    created_timestamp TIMESTAMP,
    updated_timestamp TIMESTAMP
)
USING PARQUET
PARTITIONED BY (transaction_id)
"""
create_query_in_location(query=transaction_items, location="transaction_items")

++
||
++
++



In [5]:
subscriptions = """
-- Silver: Subscriptions
CREATE TABLE IF NOT EXISTS silver.subscriptions (
    subscription_id BIGINT NOT NULL,
    customer_id BIGINT NOT NULL,
    subscription_type STRING,
    plan_id BIGINT,

    -- Dates
    start_date DATE,
    end_date DATE,
    cancellation_date DATE,
    next_billing_date DATE,

    -- Status
    status STRING,
    billing_frequency STRING,
    auto_renewal BOOLEAN,
    cancellation_reason STRING,

    -- Amounts
    subscription_amount DECIMAL(10,2),

    -- Calculated Metrics
    subscription_duration_days INT,
    total_payments_made INT,
    lifetime_value DECIMAL(12,2),

    -- Churn Analysis
    is_churned BOOLEAN,
    churn_risk_score DECIMAL(3,2),

    -- SCD Type 2
    effective_start_date TIMESTAMP,
    effective_end_date TIMESTAMP,
    is_current BOOLEAN,

    -- Metadata
    source_system STRING,
    created_timestamp TIMESTAMP,
    updated_timestamp TIMESTAMP
)
USING PARQUET
PARTITIONED BY (status, is_current)"""
create_query_in_location(query=subscriptions,location="subscriptions")

++
||
++
++



In [6]:
customer_interactions = """
CREATE TABLE IF NOT EXISTS silver.customer_interactions (
    interaction_id BIGINT NOT NULL,
    customer_id BIGINT NOT NULL,
    interaction_timestamp TIMESTAMP,
    interaction_date DATE,
    interaction_type STRING,
    channel STRING,
    agent_id BIGINT,

    -- Classification
    category STRING,
    subcategory STRING,

    -- Metrics
    sentiment_score DECIMAL(3,2),
    resolution_time_minutes INT,
    satisfaction_rating INT,

    -- Derived Fields
    interaction_year INT,
    interaction_month INT,
    is_resolved BOOLEAN,
    is_escalated BOOLEAN,

    -- Metadata
    source_system STRING,
    created_timestamp TIMESTAMP
)
USING PARQUET
PARTITIONED BY (interaction_year, interaction_month)
"""
create_query_in_location(query=customer_interactions, location="customer_interactions")

++
||
++
++



In [7]:
product_catalog = """
CREATE TABLE IF NOT EXISTS silver.product_catalog (
    product_id BIGINT NOT NULL,
    product_name STRING,

    -- Categories
    category_level1 STRING,
    category_level2 STRING,
    category_level3 STRING,
    full_category_path STRING,

    -- Brand Info
    brand STRING,
    manufacturer STRING,

    -- Pricing
    unit_cost DECIMAL(10,2),
    list_price DECIMAL(10,2),
    margin_percentage DECIMAL(5,2),
    price_tier STRING,

    -- Supply Chain
    supplier_id BIGINT,
    lead_time_days INT,
    weight_kg DECIMAL(8,3),

    -- Product Lifecycle
    launch_date DATE,
    discontinuation_date DATE,
    is_active BOOLEAN,
    product_age_days INT,
    lifecycle_stage STRING,  -- introduction, growth, maturity, decline

    -- Tags and Attributes
    tags ARRAY<STRING>,

    -- SCD Type 2
    effective_start_date TIMESTAMP,
    effective_end_date TIMESTAMP,
    is_current BOOLEAN,

    -- Metadata
    source_system STRING,
    created_timestamp TIMESTAMP,
    updated_timestamp TIMESTAMP
)
USING PARQUET
PARTITIONED BY (category_level1, is_current)
"""
create_query_in_location(query=product_catalog, location="product_catalog")

++
||
++
++



In [8]:
inventory_snapshots = """
CREATE TABLE IF NOT EXISTS silver.inventory_snapshots (
    snapshot_id BIGINT NOT NULL,
    snapshot_timestamp TIMESTAMP,
    snapshot_date DATE,
    product_id BIGINT NOT NULL,
    variant_id BIGINT,
    warehouse_id BIGINT NOT NULL,

    -- Quantities
    quantity_on_hand INT,
    quantity_reserved INT,
    quantity_available INT,
    reorder_point INT,
    reorder_quantity INT,

    -- Calculated Metrics
    days_of_supply INT,
    stock_status STRING,  -- in_stock, low_stock, out_of_stock, overstock
    is_stockout BOOLEAN,
    stockout_duration_days INT,

    -- Metadata
    source_system STRING,
    created_timestamp TIMESTAMP
)
USING PARQUET
PARTITIONED BY (snapshot_date, warehouse_id)
"""
create_query_in_location(query=inventory_snapshots, location="inventory_snapshots")

++
||
++
++



In [9]:
spark.sql("SHOW TABLES IN silver").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|   silver|customer_interact...|      false|
|   silver| inventory_snapshots|      false|
|   silver|     product_catalog|      false|
|   silver|       subscriptions|      false|
|   silver|   transaction_items|      false|
|   silver|        transactions|      false|
+---------+--------------------+-----------+

